
# Project 2: swarming behavior and correlated velocities

In this project you will explore the properties of the [Vicsek swarming model](https://en.wikipedia.org/wiki/Vicsek_model) and you will generalize it by adding rules to the interactions of the active particles in the system.

The model has an internal noise parameter ("$eta$" in the implementation below), and it is known that increasing $eta$ from $0$ to $1$ will lead the swarm to undergo a phase transition, i.e. a qualitative change of behavior, which in this case manifests itself as a passage from collective, ordered motion to random motion for the active particles. 

Referring to the concepts of polarization, correlation function, and correlation length defined in the article [Scale-free correlations in starling flocks](https://www.pnas.org/content/107/26/11865), answer the following questions:


(1) Estimate the value of $eta$ for which there is a phase transition in the Vicsek model, by using the polarization $\Phi$ of the system. Test your method for several values of size $N$ of the system, and several choices of initial random distributions of the active particles.


(2) Estimate the correlation function $C(r)$ of fluctuations of velocities within the Vicsek model, for multiple values of $eta$ and for $N \approx 10^3$. Estimate $\bar r$, the value of $r$ for which $C(r)$ becomes zero ($\bar r$ is an estimate of the correlation length of the swarm). 


(3) Research the literature to find other swarming and flocking models. On the basis of your literature review and your intuition, try to modify the Vicsek model so that you increase the average value of $\bar r$ for the swarm.  

N.B. all your answers should be in reference to the system in an "asymptotic" state when the initial random distribution of the positions and velocities of the active particles is no more significant to the dynamics.



This code template is taken from a blog post of [Francesco Turci](https://francescoturci.net/2020/06/19/minimal-vicsek-model-in-python/), a researcher in disordered systems. This specific Python implementation allows you to simulate far more active particles than it would be possible with a direct, naive implementation of the Vicsek model.

In [1]:
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib notebook

L = 32.0
rho = 3.0
N = int(rho*L**2)
print(" N",N)
 
r0 = 1.0
deltat = 1.0
factor = 0.5
v0 = r0/deltat*factor
iterations = 1000
eta = 0

pos = np.random.uniform(0,L,size=(N,2))
orient = np.random.uniform(-np.pi, np.pi,size=N)

#Sets initial pos and orient values in this cell so they cannot accidentally be altered in later versions
pos0 = pos
orient0 = orient

 N 3072


In [2]:
def Vicsek(pos, orient, eta):
    
    tree = cKDTree(pos,boxsize=[L,L])
    dist = tree.sparse_distance_matrix(tree, max_distance=r0,output_type='coo_matrix')
 
    data = np.exp(orient[dist.col]*1j)
    neigh = sparse.coo_matrix((data,(dist.row,dist.col)), shape=dist.get_shape())
    S = np.squeeze(np.asarray(neigh.tocsr().sum(axis=1)))
        
    orient = np.angle(S)+eta*np.random.uniform(-np.pi, np.pi, size=N)
        
    cos, sin= np.cos(orient), np.sin(orient)
    pos[:,0] += cos*v0
    pos[:,1] += sin*v0

    pos[pos>L] -= L
    pos[pos<0] += L
    
    return pos, orient

In [3]:
def Polarization(orient, N):
    v = (np.cos(orient), np.sin(orient))
    v_sum = np.sum(v, axis=1)
    v_mod = v_sum * (1/N)
    phi = np.linalg.norm(v_mod)
    return phi

In [15]:
def Correlation(r):
    global pos, orient, N, delta_r
    
    #Get v_mod () and v_mean ()
    v = (np.cos(orient), np.sin(orient))
    v_sum = np.sum(v, axis=1)
    v_mod = v_sum * (1/N)
    v_mean = np.mean(v)
    
    sumtop = 0
    sumbot = 0
    
    distances = np.zeros(1)

    #ui = x-component of v_mod, uj = y-component of v_mod
    ui = v_mod[0] - v_mean
    uj = v_mod[1] - v_mean
    
    for i in range(N):
        for j in range(N):
            #Get distance between two positions
            pos_1 = pos[i]
            pos_2 = pos[j]
            
            pos_1x = pos_1[0]
            pos_1y = pos_1[1]
            pos_2x = pos_2[0]
            pos_2y = pos_2[1]
            
            sub_x = (pos_1x - pos_2x)**2
            sub_y = (pos_1y - pos_2y)**2
            
            r_ij = (sub_x + sub_y)**0.5
            
            r_test = np.abs(r_ij - r)
            
            #Determine if distance is significant enough to add the dot project of ui/uj to the sum
            if r_test < delta_r:
                sumtop += np.dot(ui, uj)
                sumbot += 1
    
    Cr = sumtop / sumbot
    
    print("Count:",count)
    print("Cr for r =",r,":",Cr)
    
    return Cr

In [ ]:
"""
times = 21
count = 0
r = 0.5
delta_r = 0.01
eta_values = np.linspace(0, 1, times)

polarization_points = np.zeros(times)
correlation_points = np.zeros(times)

for eta in eta_values:
    pos = pos0
    orient = orient0
    for i in range(iterations):
        pos, orient = Vicsek(pos, orient, eta)
    polarization_points[count] = Polarization(orient, N)
    correlation_points[count] = Correlation(r)
    count += 1
"""

In [6]:
times = 5 
count = 0
eta_values = np.linspace(0, 1, times)

r = 0
delta_r = 0.03
r_values = np.linspace(0, 1, times)
r_points = np.zeros(times)
r_count = 0

polarization_points = np.zeros(times)
correlation_points = np.zeros(times)

for eta in eta_values:
    pos = pos0
    orient = orient0
    for i in range(iterations):
        pos, orient = Vicsek(pos, orient, eta)
    polarization_points[count] = Polarization(orient, N)
    for r in r_values:
        r_points[r_count] = Correlation(r)
        r_count += 1
    print("r_points for eta =",eta,":",r_points)
    count += 1
    r = 0
    r_count = 0

r_points for eta = 0.0 : [-0.49850934 -0.49850934 -0.49850934 -0.49850934 -0.49850934]


KeyboardInterrupt: 

In [ ]:
    if eta == 0:
        fig1, ax1 = plt.subplots(1, 1, figsize=(8, 8))
        plt.xlabel("r")
        plt.ylabel("Correlation")
        ax1.plot(r_values, r_points, "ro-", color = "black")
    if eta == 0.5:
        fig2, ax2 = plt.subplots(1, 1, figsize=(8, 8))
        plt.xlabel("r")
        plt.ylabel("Correlation")
        ax2.plot(r_values, r_points, "ro-", color = "black")
    if eta == 1:
        fig3, ax3 = plt.subplots(1, 1, figsize=(8, 8))
        plt.xlabel("r")
        plt.ylabel("Correlation")
        ax3.plot(r_values, r_points, "ro-", color = "black")

In [ ]:
fig1, ax1 = plt.subplots(1, 1, figsize=(8, 8))
ax1.plot(eta_values, polarization_points, "ro-", color = "black")
plt.xlabel("Eta")
plt.ylabel("Polarization")

In [ ]:
fig2, ax2 = plt.subplots(1, 1, figsize=(8, 8))
plt.xlabel("Eta")
plt.ylabel("Correlation")
ax2.plot(eta_values, correlation_points, "ro-", color = "black")

In [11]:
r = 0
delta_r = 0.03
times = 10
r_values = np.linspace(0, 1, times)
r_points = np.zeros(times)
r_count = 0

for r in r_values:
    r_points[r_count] = Correlation(r)
    r_count += 1

print(r_points)

r_test for r = 0.0 : 19.058499552501207


KeyboardInterrupt: 

In [16]:
zpz = Correlation(0)

Count: 1
Cr for r = 0 : -0.14670012874320423


In [ ]:
#Citation: Professor Napoletani

jump = np.abs(polarization_points[1:]-polarization_points[:-1])
mm = np.mean(jump[-20:])

posMax = np.argmax(jump)
inde1 = posMax

Djump = jump[inde1]
while Djump > mm:
    Djump = jump[inde1]
    inde1 += 1
    
print("Estimated eta value for phase transition:", eta_values[inde1+1])

In [ ]:
#Citation: Professor Napoletani

jump = np.abs(correlation_points[1:]-correlation_points[:-1])
mm = np.mean(jump[-20:])

posMax = np.argmax(jump)
inde1 = posMax

Djump = jump[inde1]
while Djump > mm:
    Djump = jump[inde1]
    inde1 += 1

print("Estimated eta value for correlation function becoming 0:", eta_values[inde1+1])

In [ ]:
i = 0
j = 1

pos_1 = pos[i]
pos_2 = pos[j]

pos_1x = pos_1[0]
pos_1y = pos_1[1]
pos_2x = pos_2[0]
pos_2y = pos_2[1]


print(pos[j])

